In [1]:
def query_generator(county_fips, time):
    query = ("""
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX covid-obs-property: <http://covid.geog.ucsb.edu/lod/observedproperty/>
PREFIX covid-instant: <http://covid.geog.ucsb.edu/lod/instant/>
PREFIX covid-place: <http://covid.geog.ucsb.edu/lod/place/>
PREFIX covid: <http://covid.geog.ucsb.edu/lod/ontology/>
PREFIX covid-method: <http://covid.geog.ucsb.edu/lod/method/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>


select ?modelName (min(abs(?predict_value-?groundtruth_value)) as ?min_gap) where {
     
  ?target a covid:Target ;
            sosa:observedProperty covid-obs-property:cum_death ;
            sosa:phenomenonTime ?time ;
            covid:point ?predict_value ;
            sosa:hasFeatureOfInterest ?place ;
            ^sosa:hasMember/^sosa:hasMember/sosa:madeBySensor ?model ;
     .
   
  ?model rdfs:label ?modelName .
            
  ?ground_truth a covid:GroundTruth ;
              sosa:hasFeatureOfInterest ?place ;
              sosa:observedProperty covid-obs-property:cum_death ;
              sosa:phenomenonTime ?time ;
              covid:point ?groundtruth_value .
  
  ?place covid:placeFIPS '%s' .
    
  ?time time:inXSDDateTime '%s' .
}
group by ?modelName
order by ?min_gap
""")% (county_fips, time)
    return query


### This function organizes the queries results from query_generator(model_url, date_str)

def extract_results(result):
    result_list = []
    
    for item in result['results']['bindings']:
        result_list.append([item['modelName']['value'], item['min_gap']['value']])
    
    return result_list

In [2]:
import pandas as pd
state_fips = pd.read_csv("./fips-codes/state_fips_master.csv")
state_fips['fips'] = state_fips['fips'].apply(lambda x: str(x).zfill(2))
state_fips

,state_name,state_abbr,long_name,fips,sumlev,region,division,state,region_name,division_name
0,Alabama,AL,Alabama AL,01,40,3,6,1,South,East South Central
1,Alaska,AK,Alaska AK,02,40,4,9,2,West,Pacific
2,Arizona,AZ,Arizona AZ,04,40,4,8,4,West,Mountain
3,Arkansas,AR,Arkansas AR,05,40,3,7,5,South,West South Central
4,California,CA,California CA,06,40,4,9,6,West,Pacific
5,Colorado,CO,Colorado CO,08,40,4,8,8,West,Mountain
6,Connecticut,CT,Connecticut CT,09,40,1,1,9,Northeast,New England
7,Delaware,DE,Delaware DE,10,40,3,5,10,South,South Atlantic
8,Florida,FL,Florida FL,12,40,3,5,12,South,South Atlantic
9,Georgia,GA,Georgia GA,13,40,3,5,13,South,South Atlantic


In [3]:
import pandas as pd
from pymantic import sparql
import csv
import plotly.express as px
import csv

fips_state_list = state_fips['fips'].tolist()
result_out = []
for item in fips_state_list:
    print("processing county:%s"%item)    
    query_item = query_generator(item, "2021-01-09")
    server = sparql.SPARQLServer('http://128.111.106.227:7201/repositories/Covid-KG')
    result = server.query(query_item)
    result_list = extract_results(result)
    result_pd = pd.DataFrame(result_list, columns =['Model', 'Error'])  
    OW_index = result_pd.index[result_pd['Model'] == 'OliverWyman-Navigator'].tolist()[0]
    OW_ratio = (OW_index+1)/len(result_pd)  # index start from 0, so +1
    result_out.append([item, OW_index+1, len(result_pd), OW_ratio])

processing county:01
processing county:02
processing county:04
processing county:05
processing county:06
processing county:08
processing county:09
processing county:10
processing county:12
processing county:13
processing county:15
processing county:16
processing county:17
processing county:18
processing county:19
processing county:20
processing county:21
processing county:22
processing county:23
processing county:24
processing county:25
processing county:26
processing county:27
processing county:28
processing county:29
processing county:30
processing county:31
processing county:32
processing county:33
processing county:34
processing county:35
processing county:36
processing county:37
processing county:38
processing county:39
processing county:40
processing county:41
processing county:42
processing county:44
processing county:45
processing county:46
processing county:47
processing county:48
processing county:49
processing county:50
processing county:51
processing county:53
processing co

In [4]:
result_out_pd = pd.DataFrame(result_out, columns =['state_fips', 'rank', 'total', 'ratio']) 

In [6]:
result_out_pd

,state_fips,rank,total,ratio
0,01,18,46,0.391304
1,02,19,44,0.431818
2,04,20,46,0.434783
3,05,23,46,0.500000
4,06,1,47,0.021277
5,08,5,46,0.108696
6,09,3,46,0.065217
7,10,13,48,0.270833
8,12,7,48,0.145833
9,13,10,48,0.208333


In [21]:
def convert2class(value):
    if value <=0.1:
        return "Top 10%"
    elif value>0.1 and value<=0.3:
        return "Top 30%"
    elif value>0.3 and value <=0.5:
        return "Top 50%"
    elif value>0.5 and value <=0.7:
        return "Top 70%"
    elif value>0.7 and value <=0.9:
        return "Top 90%"
    #elif value>0. and value <=0.9:
    #    return "Top 80%"
    else:
        return "Top 100%"

In [22]:
result_out_pd['class'] = result_out_pd['ratio'].apply(lambda row: convert2class(row))

In [23]:
import pandas as pd
state_fips = pd.read_csv("./fips-codes/state_fips_master.csv")
state_fips['fips'] = state_fips['fips'].apply(lambda x: str(x).zfill(2))


state_fips_merged = state_fips.merge(result_out_pd, left_on='fips', right_on='state_fips')

In [24]:
state_fips_merged_sorted = state_fips_merged.sort_values(by=['class'])

In [25]:
fig = px.choropleth(state_fips_merged_sorted, locations='state_abbr', locationmode="USA-states", color='class', 
                    scope="usa",
                   #color_discrete_sequence = px.colors.qualitative.Light24
                   color_discrete_sequence = ['green', 'lightgreen', 'yellow', 'gold', '#FD3216','' ],
                   )

fig.update_layout(
    title_text = 'Where Does OW-Navigator Model Work the Best in the U.S. on Forecasting Cum Death on Jan 9 2021?'
)
fig.show()

In [15]:
import plotly.figure_factory as ff

import numpy as np
import pandas as pd

#colorscale = ['#FD3216','gold','yellow','lightgreen','green']
colorscale = ['darkgreen','green', 'lime', 'gold','yellow', 'red', 'darkred']

endpts = list(np.linspace(0, 1, len(colorscale) - 1))

fips = state_fips_merged['state_fips'].tolist()
values = state_fips_merged['ratio'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Where Does OW-Navigator Model Work the Best in the U.S. on Forecasting Inc. Case on Jan 9 2021?',
    legend_title='Class'
)

fig.layout.template = None
fig.show()

/home/rui/anaconda3/lib/python3.7/site-packages/plotly/figure_factory/_county_choropleth.py:838: UserWarning:

Unrecognized FIPS Values

Whoops! It looks like you are trying to pass at least one FIPS value that is not in our shapefile of FIPS and data for the counties. Your choropleth will still show up but these counties cannot be shown.
Unrecognized FIPS are: [1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56]

